In [1]:
!pip install pyodbc pandas sqlalchemy

In [2]:
import pyodbc
import pandas as pd
from sqlalchemy import create_engine, text

In [3]:
server = 'ANJALI\SQLEXPRESS22'  # e.g., 'localhost\SQLEXPRESS' or 'YourServerName'
database = 'Walmart'
# username = 'YourUsername'  # If using SQL Server Authentication
# password = 'YourPassword'  # If using SQL Server Authentication
driver = '{ODBC Driver 17 for SQL Server}' # Or '{SQL Server}' or '{SQL Server Native Client 11.0}'

In [4]:
    connection_string = (
        f"DRIVER={driver};"
        f"SERVER={server};"
        f"DATABASE={database};"
        f"Trusted_Connection=yes;"
    )

In [27]:
    try:
        cnxn = pyodbc.connect(connection_string)
        cursor = cnxn.cursor()
        print("Connection successful!")
    except pyodbc.Error as ex:
        sqlstate = ex.args[0]
        print(f"Error connecting to SQL Server: {sqlstate}")

Connection successful!


In [24]:
    try:
        query = "SELECT TOP 10 * FROM Customers;" # Replace YourTable with your table name
        cursor.execute(query)
        rows = cursor.fetchall()
        for row in rows:
            print(row)
    except pyodbc.Error as ex:
        sqlstate = ex.args[0]
        print(f"Error executing query: {sqlstate}")
    finally:
        if 'cnxn' in locals() and cnxn:
            cnxn.close()
            print("Connection closed.")

('00012a2ce6f8dcda20d059ce98491703', '248ffe10d632bebe4f7267f1f44844c9', 6273, 'osasco                                                                                              ', 'SP   ')
('000161a058600d5901f007fab4c27140', 'b0015e09bb4b6e47c52844fab5fb6638', 35550, 'itapecerica                                                                                         ', 'MG   ')
('0001fd6190edaaf884bcaf3d49edf079', '94b11d37cd61cb2994a194d11f89682b', 29830, 'nova venecia                                                                                        ', 'ES   ')
('0002414f95344307404f0ace7a26f1d5', '4893ad4ea28b2c5b3ddf4e82e79db9e6', 39664, 'mendonca                                                                                            ', 'MG   ')
('000379cdec625522490c315e70c7a9fb', '0b83f73b19c2019e182fd552c048a22c', 4841, 'sao paulo                                                                                           ', 'SP   ')
('0004164d20a9e969af783496f3408652', 

In [28]:
    try:
        query = "SELECT * FROM Customers;"
        df = pd.read_sql(query, cnxn)
        cnxn.close()
        print("Data loaded successfully into DataFrame.")
        print(df)
    except pyodbc.Error as ex:
        sqlstate = ex.args[0]
        print(f"Error connecting to SQL Server or loading data: {sqlstate}")

C:\Users\anjal\AppData\Local\Temp\ipykernel_22684\1176961082.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, cnxn)


Data loaded successfully into DataFrame.
                            customer_id                customer_unique_id  \
0      00012a2ce6f8dcda20d059ce98491703  248ffe10d632bebe4f7267f1f44844c9   
1      000161a058600d5901f007fab4c27140  b0015e09bb4b6e47c52844fab5fb6638   
2      0001fd6190edaaf884bcaf3d49edf079  94b11d37cd61cb2994a194d11f89682b   
3      0002414f95344307404f0ace7a26f1d5  4893ad4ea28b2c5b3ddf4e82e79db9e6   
4      000379cdec625522490c315e70c7a9fb  0b83f73b19c2019e182fd552c048a22c   
...                                 ...                               ...   
99436  fffecc9f79fd8c764f843e9951b11341  e5794df8573fa179a90a7b797fc4b71f   
99437  fffeda5b6d849fbd39689bb92087f431  afbb5a642107cf6bb1ca68e863175f03   
99438  ffff42319e9b2d713724ae527742af25  680213db6ebd9e4f24d03280cbe10346   
99439  ffffa3172527f765de70084a7e53aae8  48fd7dec70f2b104a1d5e8c5c639102b   
99440  ffffe8b65bbe3087b653a978c870db99  736e6bfa0510aa5b878881a226a5fd89   

       customer_zip_code_prefix  \

In [34]:
from sqlalchemy import create_engine
import pandas as pd

# Define your server and database names
server = 'ANJALI\SQLEXPRESS22'  # or 'YOUR_SERVER\\SQLEXPRESS'
database = 'Walmart'

# Create connection string using trusted connection
engine_str = f"mssql+pyodbc://@{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes"

# Create SQLAlchemy engine
engine = create_engine(engine_str)

# Step 2: List of table names
table_names = [
    'Customers', 'Geolocations', 'Orders', 'Products',
    'Walmart_Trustpilot_Reviews', 'Sellers', 'Payments', 'Order_items'
]

# Step 3: Define a function for basic cleaning
def clean_dataframe(df):
    df_columns = [col.strip().lower().replace(" ", "_") for col in df.columns]  # Standardize column names
    df_duplicates = df.drop_duplicates()  # Remove duplicate rows
    df_cleaned = df.dropna()  # Remove rows with any null values
    return df

# Step 4: Load, clean, and overwrite each table
for table in table_names:
    try:
        print(f"\nProcessing table: {table}")
        df = pd.read_sql(f"SELECT * FROM {table}", engine)
        df_clean = clean_dataframe(df)
        print(f"Cleaned rows: {len(df_clean)}")

        # Overwrite the original table with cleaned data
        df_clean.to_sql(name=f"Cleaned_{table}", con=engine, index=False, if_exists='replace')
        print(f"✅ {table} overwritten in database with cleaned data.")

    except Exception as e:
        print(f"❌ Error processing {table}: {e}")
    
'''
# SQL query to execute
query = "SELECT * FROM Customers"

# Load result into DataFrame
df = pd.read_sql(query, engine)

# Display the DataFrame
print(df)
'''


Processing table: Customers
Cleaned rows: 99441
✅ Customers overwritten in database with cleaned data.

Processing table: Geolocations
Cleaned rows: 19015
✅ Geolocations overwritten in database with cleaned data.

Processing table: Orders
Cleaned rows: 99441
✅ Orders overwritten in database with cleaned data.

Processing table: Products
Cleaned rows: 32951
✅ Products overwritten in database with cleaned data.

Processing table: Walmart_Trustpilot_Reviews
Cleaned rows: 3671
✅ Walmart_Trustpilot_Reviews overwritten in database with cleaned data.

Processing table: Sellers
Cleaned rows: 3095
✅ Sellers overwritten in database with cleaned data.

Processing table: Payments
Cleaned rows: 103886
✅ Payments overwritten in database with cleaned data.

Processing table: Order_items
Cleaned rows: 112650
✅ Order_items overwritten in database with cleaned data.


'\n# Create SQLAlchemy engine\nengine = create_engine(engine_str)\n\n# SQL query to execute\nquery = "SELECT * FROM Customers"\n\n# Load result into DataFrame\ndf = pd.read_sql(query, engine)\n\n# Display the DataFrame\nprint(df)\n'